In [ ]:
using Pkg
Pkg.activate(".")

In [ ]:
using HypergraphModularity
using LinearAlgebra
using Printf
using SparseArrays
using StatsBase

In [ ]:
function all_or_nothing(p::Vector{Int64})
    is_aon = length(p) == 1
    return (sum(p), is_aon)
end

function polyadic_AON_MLE(Hyp, Z)
    Ω̂ = estimateΩEmpirically(Hyp, Z; min_val=0, aggregator=all_or_nothing)
    ll = Float64(sum(logLikelihood(Hyp, Z, Ω̂; α=0, bigInt=true)))
    return (ll, Ω̂)    
end

function dyadic_MLE(Hyp, Z, weighted::Bool, binary::Bool)
    ω_in, ω_out = 
        computeDyadicResolutionParameter(Hyp, Z; mode="ω", weighted=weighted, binary=binary)
    γ = (ω_in - ω_out) / (log(ω_in) - log(ω_out))
    Q = dyadicModularity(Hyp, Z, γ; weighted=weighted, binary=binary)
    ll = Float64(dyadicLogLikelihood(Hyp, Z, ω_in, ω_out; weighted=weighted, binary=binary))
    return (ll, ω_in, ω_out, γ, Q)
end

In [ ]:
function decomposedDyadicModularity(H, Z, γ, weighted, binary)
    G = CliqueExpansion(H, weighted, binary) 
    cluster_ids = unique(Z)
    
    # edge terms
    edges = Dict{Int64, Float64}(c => 0.0 for c in cluster_ids)
    for (i, j, v) in zip(SparseArrays.findnz(G)...)
        if Z[i] == Z[j]
            edges[Z[i]] += v
        end
	end

    # volume terms                                                                                                                                                                                                
    d = vec(sum(G, dims=1))    
    vols = Dict{Int64, Float64}(c => 0.0 for c in cluster_ids)
    for (ci, di) in zip(Z, d)
        vols[ci] += di
    end

    # modularity terms
    volG = sum(d)
    Qs = Dict{Int64, Float64}(c => 0.0 for c in cluster_ids)
    for c in cluster_ids
        Qs[c] = (edges[c] - γ * vols[c]^2 / volG) / volG
    end

    return Qs
end

In [ ]:
function decomposedAONPolyadicModularity(H, Z, Ω)
    cluster_ids = unique(Z)
    all_Qs = Dict()
    combined_Qs = Dict{Int64, Float64}(c => 0.0 for c in cluster_ids)
    
    for ℓ = 2:maximum(keys(H.E))
        if haskey(H.E, ℓ)
            # Volume and W terms
            vols = Dict{Int64, Float64}(c => 0.0 for c in cluster_ids)
            Ws = Dict{Int64, Float64}(c => 0.0 for c in cluster_ids)
            for (edge, weight) in H.E[ℓ]
                Zedge = [Z[v] for v in edge]
                if all(z -> z == Zedge[1], Zedge)
                   Ws[Zedge[1]] += weight
                end
                for c in Zedge
                   vols[c] += weight
                end
            end
            
            # Modularities
            ω1 = Ω.ω((ℓ, true),  0)
            ω0 = Ω.ω((ℓ, false), 0)
            α = log(ω1) - log(ω0)
            β = ω1 - ω0
            Qs = Dict{Int64, Float64}(c => 0.0 for c in cluster_ids)
            for c in cluster_ids
                Qs[c] = α * Ws[c] - β * vols[c]^ℓ
                combined_Qs[c] += Qs[c]
            end
            
            all_Qs[ℓ] = Qs
        end
    end
    
    return all_Qs, combined_Qs
end

In [ ]:
function dyadic_modularity_contributions(H, labels, names, topk=10, weighted=false, binary=true)
    g_ll, ω_in, ω_out, γ, Q = dyadic_MLE(H, labels, weighted, binary)
    dyadicQs = decomposedDyadicModularity(H, labels, γ, weighted, binary)
    stopind = min(topk, length(names))

    # Dyadic from clique expansion
    println("dyadic...")
    dyadicQ = sort([(name, dyadicQs[i]) for (i, name) in enumerate(names)], by=kv->kv[2], rev=true)[1:stopind]
    for (i, (name, Q)) in enumerate(dyadicQ)
        println(rpad("$i: ", 5), 
                rpad(name, 31),
                rpad(@sprintf("%.3E", Q), 15))
    end
    println("------")
end

function polyadic_modularity_contributions(H, labels, names, topk=10, individual=false)
    h_ll, Ω̂ = polyadic_AON_MLE(H, labels)
    Qs, combined_Qs = decomposedAONPolyadicModularity(H, labels, Ω̂)
    stopind = min(topk, length(names))
   
    # Polyadic (combined)
    println("polyadic...")
    polyadicQ = sort([(name, combined_Qs[i]) for (i, name) in enumerate(names)], by=kv->kv[2], rev=true)[1:stopind]
    for (i, (name, Q)) in enumerate(polyadicQ)
        println(rpad("$i: ", 5), 
                rpad(name, 31),
                rpad(@sprintf("%.3E", Q), 15))
    end
    println("------")
    
    if individual
        # Polyadic (per hyperedge size)
        for key in sort(collect(keys(Qs)))
            println("ℓ = $(key)...")
            Qℓ = Qs[key]
            order_ℓ = sort([(name, Qℓ[i]) for (i, name) in enumerate(names)], by=kv->kv[2], rev=true)[1:stopind]
            for (i, (name, Q)) in enumerate(order_ℓ)
                println(rpad("$i: ", 5), 
                        rpad(name, 31),
                        rpad(@sprintf("%.3E", Q), 15))
            end
            println("------")
        end
    end
end

In [ ]:
dataset = "walmart-trips"
H, labels = read_hypergraph_data(dataset, 10)
names = read_hypergraph_label_names(dataset)

# throw out "Other" category
other_key = "Other"
other_ind = findfirst(names .== other_key)
keep = Vector{Bool}(labels .!= other_ind)
(subH, node_map) = subhypergraph(H, keep)
sub_labels = zeros(Int64, maximum(subH.N))
for (i, j) in node_map
    sub_labels[j] = labels[i]
end

# Just to keep rest of the code the same
H = subH
labels = sub_labels
names = names[names .!= other_key]
;

In [ ]:
dyadic_modularity_contributions(H, labels, names)
polyadic_modularity_contributions(H, labels, names)

In [ ]:
dataset = "TrivagoClickout"
H, labels = read_hypergraph_data(dataset, 5)
names = read_hypergraph_label_names(dataset)
dyadic_modularity_contributions(H, labels, names)
polyadic_modularity_contributions(H, labels, names)

In [ ]:
dataset = "congress-bills"
H, labels = read_hypergraph_data(dataset, 10)
names = read_hypergraph_label_names(dataset)
dyadic_modularity_contributions(H, labels, names)
polyadic_modularity_contributions(H, labels, names)

In [ ]:
dataset = "contact-high-school-classes"
H, labels = read_hypergraph_data(dataset, 10)
names = read_hypergraph_label_names(dataset)
dyadic_modularity_contributions(H, labels, names)
polyadic_modularity_contributions(H, labels, names)

In [ ]:
dataset = "contact-primary-school-classes"
H, labels = read_hypergraph_data(dataset, 10)
names = read_hypergraph_label_names(dataset)
;

In [ ]:
println("Weight-normalized")
dyadic_modularity_contributions(H, labels, names, 12, true, false)
println("Clique-weighted")
dyadic_modularity_contributions(H, labels, names, 12, false, false)
println("binary")
dyadic_modularity_contributions(H, labels, names, 12, false, true)
polyadic_modularity_contributions(H, labels, names, 12)

In [ ]:
for k = 2:5
    H, labels = read_hypergraph_data(dataset, k, k)
    names = read_hypergraph_label_names(dataset)
    println("$k...")
    dyadic_modularity_contributions(H, labels, names)
end